In [ ]:
import sys

assert sys.version_info >= (3, 7)


In [ ]:
from packaging import version
import sklearn

assert version.parse(sklearn.__version__) >= version.parse("1.0.1")

In [ ]:
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1" 

import tf_keras


In [ ]:
import tensorflow as tf

assert version.parse(tf.__version__) >= version.parse("2.8.0")

In [ ]:
import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

In [ ]:
from pathlib import Path

IMAGES_PATH = Path() / "images" / "generative"
IMAGES_PATH.mkdir(parents=True, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = IMAGES_PATH / f"{fig_id}.{fig_extension}"
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [ ]:
import numpy as np

In [ ]:
import random

In [ ]:
import os
from pathlib import Path
import numpy as np
from PIL import Image
from skimage.transform import resize

In [ ]:
import os
import numpy as np
from PIL import Image
from pathlib import Path
from sklearn.model_selection import train_test_split

## Define paths:

In [ ]:
os.getcwd()

In [ ]:
#root_path = Path('/Users/stephanehess/Documents/CAS_AML/dias_digit_project/project')

In [ ]:
project_path = Path.cwd()
root_path = (project_path / '..').resolve()

In [ ]:
image_dir_1 = root_path/"data_1"

In [ ]:
image_dir_1

## Load and process all images:

In [ ]:
#os.listdir(root_path)

In [ ]:
os.listdir(image_dir_1)

In [ ]:
for image_path in Path(image_dir_1).glob("*.tif"):
    print(image_path)
    break

In [ ]:
def load_process_images(image_dir, validation_split=0.2):
    image_arrays = []
    image_identifiers = []

    # Determine the smallest square size
    target_size = None
    for image_path in Path(image_dir).glob("*.tif"):
        try:
            with Image.open(image_path) as img:
                width, height = img.size
                min_dim = min(width, height)
                if target_size is None or min_dim < target_size:
                    target_size = min_dim
        except Exception as e:
            print(f"Error determining size for {image_path}: {e}")

    if target_size is None:
        raise ValueError("No valid images found in the directory.")

    # Process and resize all images
    for image_path in Path(image_dir).glob("*.tif"):
        try:
            with Image.open(image_path) as img:
                # Convert to grayscale
                img = img.convert("L")  # 'L' mode ensures single-channel grayscale

                # Extract dimensions
                width, height = img.size

                # Calculate cropping box to center the crop
                top = (height - target_size) // 2
                left = (width - target_size) // 2
                bottom = top + target_size
                right = left + target_size

                # Crop and resize the image to ensure square dimensions
                img_cropped = img.crop((left, top, right, bottom))
                img_resized = img_cropped.resize((target_size, target_size))
                
                # Convert to numpy array and append to list
                image_arrays.append(np.array(img_resized))
                
                
                # Extract identifier from the last three characters of the file name
                identifier = image_path.stem[-3:]
                image_identifiers.append(identifier)

        except Exception as e:
            print(f"Error processing {image_path}: {e}")

    # Convert to numpy array (all images are guaranteed to have the same size)
    image_arrays = np.array(image_arrays)

    # Shuffle and split into training and validation sets
    data_indices = list(range(len(image_arrays)))
    random.shuffle(data_indices)

    val_count = int(len(image_arrays) * validation_split)
    val_indices = data_indices[:val_count]
    train_indices = data_indices[val_count:]

    train_data = image_arrays[train_indices]
    val_data = image_arrays[val_indices]

    train_identifiers = [image_identifiers[i] for i in train_indices]
    val_identifiers = [image_identifiers[i] for i in val_indices]

    return train_data, val_data, train_identifiers, val_identifiers

# Process images and split into training/validation sets
train_images, val_images, train_ids, val_ids = load_process_images(image_dir_1)

# Print shapes and identifiers
print(f"Training data shape: {train_images.shape}")
print(f"Validation data shape: {val_images.shape}")
print(f"Training identifiers: {train_ids}")
print(f"Validation identifiers: {val_ids}")



In [ ]:
type(train_images)

In [ ]:
train_images.shape

## Encode one single image as the simple most possible task (using dense network):

In [ ]:
try_image = train_images[0:1]

In [ ]:
try_image.shape

In [ ]:
try_image[0]

In [ ]:
plt.imshow(try_image[0])

## Set image dimensions to be used in dense model:

In [ ]:
min_dim = try_image.shape[1]
min_dim

In [ ]:
tf.random.set_seed(42)  # extra code – ensures reproducibility on CPU

stacked_encoder = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(30, activation="relu"),
])
stacked_decoder = tf.keras.Sequential([
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(min_dim * min_dim),
    tf.keras.layers.Reshape([min_dim, min_dim])
])
stacked_ae = tf.keras.Sequential([stacked_encoder, stacked_decoder])

stacked_ae.compile(loss="mse", optimizer="nadam")                   
history = stacked_ae.fit(try_image, try_image, epochs=30,
                         validation_data=(try_image, try_image))

In [ ]:
reconstructions = stacked_ae.predict(try_image[0:1])
plt.subplot(2, 1, 1)
plt.imshow(try_image[0])
plt.axis("off")
plt.subplot(2, 1, 2)
plt.imshow(reconstructions[0])
plt.axis("off")
plt.show()

In [ ]:
stacked_ae.summary()

In [ ]:
stacked_encoder.summary()

## Find two similar images to make the task a bit more complicated:

In [ ]:
idx_1 = train_ids.index('022')

In [ ]:
idx_2 = train_ids.index('023')

In [ ]:
plt.imshow(train_images[idx_1])

In [ ]:
plt.imshow(train_images[idx_2])

In [ ]:
similar_images_1 = np.array([train_images[idx_1], train_images[idx_2]])
similar_images_1.shape

In [ ]:
tf.random.set_seed(42)  # extra code – ensures reproducibility on CPU

stacked_ae = tf.keras.Sequential([stacked_encoder, stacked_decoder])

stacked_ae.compile(loss="mse", optimizer="nadam")                   
history = stacked_ae.fit(similar_images_1, similar_images_1, epochs=30,
                         validation_data=(similar_images_1, similar_images_1))

In [ ]:
reconstructions = stacked_ae.predict(similar_images_1[0:2])
for image_idx in range(0, similar_images_1.shape[0]):
    print(image_idx)
    plt.subplot(2, 2, image_idx + 1)
    plt.imshow(similar_images_1[image_idx])
    plt.axis("off")
    plt.subplot(2, 2, image_idx + 2 + 1)
    plt.imshow(reconstructions[image_idx])
    plt.axis("off")
plt.show()

## Rebuild model with latent space of 300 and train:

In [ ]:
tf.random.set_seed(42)  # extra code – ensures reproducibility on CPU

stacked_encoder = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(500, activation="relu"),
    tf.keras.layers.Dense(300, activation="relu"),
])
stacked_decoder = tf.keras.Sequential([
    tf.keras.layers.Dense(500, activation="relu"),
    tf.keras.layers.Dense(min_dim * min_dim),
    tf.keras.layers.Reshape([min_dim, min_dim])
])
stacked_ae = tf.keras.Sequential([stacked_encoder, stacked_decoder])

stacked_ae.compile(loss="mse", optimizer="nadam")                   
history = stacked_ae.fit(similar_images_1, similar_images_1, epochs=55,
                         validation_data=(similar_images_1, similar_images_1))

In [ ]:
reconstructions = stacked_ae.predict(similar_images_1[0:2])
for image_idx in range(0, similar_images_1.shape[0]):
    print(image_idx)
    plt.subplot(2, 2, image_idx + 1)
    plt.imshow(similar_images_1[image_idx])
    plt.axis("off")
    plt.subplot(2, 2, image_idx + 2 + 1)
    plt.imshow(reconstructions[image_idx])
    plt.axis("off")
plt.show()

In [ ]:
plt.imshow(reconstructions[0])

In [ ]:
plt.imshow(reconstructions[1])

In [ ]:
from sklearn.manifold import TSNE

similar_images_1_compressed = stacked_encoder.predict(similar_images_1)

In [ ]:
type(similar_images_1_compressed)

In [ ]:
similar_images_1_compressed.shape

In [ ]:
#from sklearn.manifold import TSNE

#similar_images_1_compressed = stacked_encoder.predict(similar_images_1)
tsne = TSNE(perplexity= 1, init="pca", learning_rate="auto", random_state=42)
X_valid_2D = tsne.fit_transform(similar_images_1_compressed)

In [ ]:
plt.scatter(X_valid_2D[:, 0], X_valid_2D[:, 1], s=10, cmap="tab10")
plt.show()

## Try the same with a convolutional neural network:

In [ ]:
length_of_img_square = 572

In [ ]:
similar_images_1 = similar_images_1.astype("float32") / 255.0

In [ ]:
similar_images_1[0].shape

In [ ]:
similar_images_1.shape

In [ ]:
from functools import partial

In [ ]:
similar_images_1.shape

In [ ]:
def crop_images(image_data, crop_size):
    """
    Crops images stored in a NumPy array to the specified size.
    
    Parameters:
        image_data (numpy.ndarray): The input image data of shape (n, m, m), where n is the number of images.
        crop_size (int): The desired dimension e for the cropped images (output shape will be (n, e, e)).
        
    Returns:
        numpy.ndarray: Cropped image data of shape (n, e, e).
    """
    n, m, _ = image_data.shape  # Get number of images and dimensions of each image
    e = crop_size  # Target size (e x e)
    
    if e >= m:
        raise ValueError("Crop size must be smaller than the original image size.")
    
    # Calculate cropping margins
    start = (m - e) // 2  # Start index for cropping
    end = start + e       # End index for cropping
    
    # Crop each image
    cropped_images = image_data[:, start:end, start:end]
    
    return cropped_images

In [ ]:
9*3*4*4

In [ ]:
desired_dim = 28

In [ ]:
cropped_images = crop_images(similar_images_1, desired_dim)

In [ ]:
cropped_images.shape

In [ ]:
plt.imshow(cropped_images[0])

In [ ]:
plt.imshow(cropped_images[1])

In [ ]:
cropped_images = cropped_images.astype("float32") / 255.0

In [ ]:
cropped_images.shape

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist
X_train_full = X_train_full.astype(np.float32) / 255
X_test = X_test.astype(np.float32) / 255
X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]

In [ ]:
X_train.shape

In [ ]:
X_train_select = X_train[0:2].copy()
X_train_select.shape

In [ ]:
X_valid.shape

In [ ]:
X_valid_select = X_valid[0:500].copy()

In [ ]:
plt.imshow(X_train_select[0])

In [ ]:
plt.imshow(X_train_select[1])

In [ ]:
#experiment_data = cropped_images
experiment_data = X_train_select

tf.random.set_seed(42)  # extra code – ensures reproducibility on CPU

conv_encoder = tf.keras.Sequential([
    tf.keras.layers.Reshape([28, 28, 1]),
    tf.keras.layers.Conv2D(16, 3, padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=2),  # output: 14 × 14 x 16
    tf.keras.layers.Conv2D(32, 3, padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=2),  # output: 7 × 7 x 326
    tf.keras.layers.Conv2D(64, 3, padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=2),  # output: 3 × 3 x 64
    tf.keras.layers.Conv2D(30, 3, padding="same", activation="relu"),
    tf.keras.layers.GlobalAvgPool2D()  # output: 30
])
conv_decoder = tf.keras.Sequential([
    tf.keras.layers.Dense(3 * 3 * 16),
    tf.keras.layers.Reshape((3, 3, 16)),
    tf.keras.layers.Conv2DTranspose(32, 3, strides=2, activation="relu"),
    tf.keras.layers.Conv2DTranspose(16, 3, strides=2, padding="same",
                                    activation="relu"),
    tf.keras.layers.Conv2DTranspose(1, 3, strides=2, padding="same"),
    tf.keras.layers.Reshape([28, 28])
])
conv_ae = tf.keras.Sequential([conv_encoder, conv_decoder])

# extra code – compiles and fits the model
conv_ae.compile(loss="mse", optimizer="nadam")
history = conv_ae.fit(experiment_data, experiment_data, epochs=10,
                      validation_data=(experiment_data, experiment_data))

In [ ]:
reconstructions = conv_ae.predict(X_train_select)

In [ ]:
for image_idx in range(0, experiment_data.shape[0]):
    #print(image_idx)
    plt.subplot(2, experiment_data.shape[0], image_idx + 1)
    plt.imshow(experiment_data[image_idx], cmap="binary")
    plt.axis("off")
    plt.subplot(2, experiment_data.shape[0], image_idx + experiment_data.shape[0] + 1)
    plt.imshow(reconstructions[image_idx], cmap="binary")
    plt.axis("off")
plt.show()

In [ ]:
stacked_encoder.summary()

In [ ]:
conv_encoder.summary()

In [ ]:
conv_decoder.summary()

In [ ]:
conv_ae.summary()

## Use larger data set to showcase that the convolutional network needs training to adequately encode images:

In [ ]:

tf.random.set_seed(42)  # extra code – ensures reproducibility on CPU

conv_encoder = tf.keras.Sequential([
    tf.keras.layers.Reshape([28, 28, 1]),
    tf.keras.layers.Conv2D(16, 3, padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=2),  # output: 14 × 14 x 16
    tf.keras.layers.Conv2D(32, 3, padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=2),  # output: 7 × 7 x 326
    tf.keras.layers.Conv2D(64, 3, padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=2),  # output: 3 × 3 x 64
    tf.keras.layers.Conv2D(30, 3, padding="same", activation="relu"),
    tf.keras.layers.GlobalAvgPool2D()  # output: 30
])
conv_decoder = tf.keras.Sequential([
    tf.keras.layers.Dense(3 * 3 * 16),
    tf.keras.layers.Reshape((3, 3, 16)),
    tf.keras.layers.Conv2DTranspose(32, 3, strides=2, activation="relu"),
    tf.keras.layers.Conv2DTranspose(16, 3, strides=2, padding="same",
                                    activation="relu"),
    tf.keras.layers.Conv2DTranspose(1, 3, strides=2, padding="same"),
    tf.keras.layers.Reshape([28, 28])
])
conv_ae = tf.keras.Sequential([conv_encoder, conv_decoder])

# extra code – compiles and fits the model
conv_ae.compile(loss="mse", optimizer="nadam")
history = conv_ae.fit(X_train, X_train, epochs=10,
                      validation_data=(X_valid_select, X_valid_select))

In [ ]:
reconstructions = conv_ae.predict(X_train_select)

In [ ]:
reconstructions.shape

In [ ]:
X_train.shape

In [ ]:
X_train.shape[0]

In [ ]:
X_train_select.shape

In [ ]:
X_train_select.shape[0]

In [ ]:

for image_idx in range(0, X_train_select.shape[0]):
    #print(image_idx)
    plt.subplot(2, X_train_select.shape[0], image_idx + 1)
    plt.imshow(X_train_select[image_idx], cmap="binary")
    plt.axis("off")
    plt.subplot(2, X_train_select.shape[0], image_idx + X_train_select.shape[0] + 1)
    plt.imshow(reconstructions[image_idx], cmap="binary")
    plt.axis("off")
plt.show()

In [ ]:
plot_reconstructions(conv_ae, images=X_train_select, n_images=12)

In [ ]:


tf.random.set_seed(42)  # extra code – ensures reproducibility on CPU¨


#DefaultConv2D = partial(tf.keras.layers.Conv2D, kernel_size = 3, padding='same', 
 #                       activation = 'relu', kernel_initializer='he_normal')

conv_encoder = tf.keras.Sequential([
    tf.keras.layers.Reshape([desired_dim, desired_dim, 1]),
    tf.keras.layers.Conv2D(16, 3, padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=4),  
    tf.keras.layers.Conv2D(32, 3, padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=4),  
    tf.keras.layers.Conv2D(64, 3, padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=3),  
    tf.keras.layers.Conv2D(64, 9, padding="same", activation="relu"),
    tf.keras.layers.GlobalAvgPool2D()  # output: 30
])
conv_decoder = tf.keras.Sequential([
    tf.keras.layers.Dense(9 * 9 * 16),
    tf.keras.layers.Reshape((9, 9, 16)),
    tf.keras.layers.Conv2DTranspose(32, 3, strides=3, activation="relu"),
    tf.keras.layers.Conv2DTranspose(16, 3, strides=4, padding="same",
                                    activation="relu"),
    tf.keras.layers.Conv2DTranspose(1, 3, strides=4, padding="same"),
    tf.keras.layers.Reshape([desired_dim, desired_dim])
])
conv_ae = tf.keras.Sequential([conv_encoder, conv_decoder])

# extra code – compiles and fits the model
conv_ae.compile(loss="mse", optimizer="nadam")
history = conv_ae.fit(cropped_images, cropped_images, epochs=30,
                      validation_data=(cropped_images, cropped_images))

## Putative formula for calculating outputs of hidden layers:

In [ ]:
conv_encoder.summary()

In [ ]:
conv_decoder.summary()

In [ ]:
conv_ae.summary()

In [ ]:
reconstructions = conv_ae.predict(cropped_images[0:2])

In [ ]:
type(reconstructions)

In [ ]:
reconstructions.shape

In [ ]:
plt.imshow(reconstructions[0])

In [ ]:
reconstructions = conv_ae.predict(cropped_images[0:2])
for image_idx in range(0, cropped_images.shape[0]):
    print(image_idx)
    plt.subplot(2, 2, image_idx + 1)
    plt.imshow(cropped_images[image_idx], cmap="binary")
    plt.axis("off")
    plt.subplot(2, 2, image_idx + 2 + 1)
    plt.imshow(reconstructions[image_idx], cmap="binary")
    plt.axis("off")
plt.show()

In [ ]:
reconstructions

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist
X_train_full = X_train_full.astype(np.float32) / 255
X_test = X_test.astype(np.float32) / 255
X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]

In [ ]:
X_train.shape

In [ ]:
X_train[0].shape

In [ ]:
import numpy as np

def plot_reconstructions(model, images=X_valid, n_images=5):
    reconstructions = np.clip(model.predict(images[:n_images]), 0, 1)
    fig = plt.figure(figsize=(n_images * 1.5, 3))
    for image_index in range(n_images):
        plt.subplot(2, n_images, 1 + image_index)
        plt.imshow(images[image_index], cmap="binary")
        plt.axis("off")
        plt.subplot(2, n_images, 1 + n_images + image_index)
        plt.imshow(reconstructions[image_index], cmap="binary")
        plt.axis("off")

In [ ]:
tf.random.set_seed(42)  # extra code – ensures reproducibility on CPU

conv_encoder = tf.keras.Sequential([
    tf.keras.layers.Reshape([28, 28, 1]),
    tf.keras.layers.Conv2D(16, 3, padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=2),  # output: 14 × 14 x 16
    tf.keras.layers.Conv2D(32, 3, padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=2),  # output: 7 × 7 x 326
    tf.keras.layers.Conv2D(64, 3, padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=2),  # output: 3 × 3 x 64
    tf.keras.layers.Conv2D(30, 3, padding="same", activation="relu"),
    tf.keras.layers.GlobalAvgPool2D()  # output: 30
])
conv_decoder = tf.keras.Sequential([
    tf.keras.layers.Dense(3 * 3 * 16),
    tf.keras.layers.Reshape((3, 3, 16)),
    tf.keras.layers.Conv2DTranspose(32, 3, strides=2, activation="relu"),
    tf.keras.layers.Conv2DTranspose(16, 3, strides=2, padding="same",
                                    activation="relu"),
    tf.keras.layers.Conv2DTranspose(1, 3, strides=2, padding="same"),
    tf.keras.layers.Reshape([28, 28])
])
conv_ae = tf.keras.Sequential([conv_encoder, conv_decoder])

# extra code – compiles and fits the model
conv_ae.compile(loss="mse", optimizer="nadam")
history = conv_ae.fit(X_train, X_train, epochs=10,
                      validation_data=(X_valid, X_valid))


In [ ]:
plot_reconstructions(conv_ae)
plt.show()

In [ ]:
plt.imshow(X_train[0], cmap="binary")

In [ ]:
X_valid.shape

In [ ]:
import numpy as np

def plot_reconstructions(model, images=X_valid, n_images=5):
    reconstructions = np.clip(model.predict(images[:n_images]), 0, 1)
    fig = plt.figure(figsize=(n_images * 1.5, 3))
    for image_index in range(n_images):
        plt.subplot(2, n_images, 1 + image_index)
        plt.imshow(images[image_index], cmap="binary")
        plt.axis("off")
        plt.subplot(2, n_images, 1 + n_images + image_index)
        plt.imshow(reconstructions[image_index], cmap="binary")
        plt.axis("off")

plot_reconstructions(stacked_ae)
save_fig("reconstruction_plot")  # extra code – saves the high res figure
plt.show()

In [ ]:
from sklearn.manifold import TSNE

X_valid_compressed = stacked_encoder.predict(X_valid)
tsne = TSNE(init="pca", learning_rate="auto", random_state=42)
X_valid_2D = tsne.fit_transform(X_valid_compressed)


In [ ]:
plt.scatter(X_valid_2D[:, 0], X_valid_2D[:, 1], c=y_valid, s=10, cmap="tab10")
plt.show()